# MAPPO Integration

This runs the integrated MAPPO algorithm with the patrolling zoo.

In [1]:
%reload_ext autoreload
%autoreload 2
from onpolicy.scripts.train.train_patrolling import get_config, parse_args, main

import os
os.environ["WANDB__SERVICE_WAIT"] = "300"

In [2]:
parser = get_config()
all_args = parse_args([], parser)

all_args.experiment_name = "gnnMultiLayerNoMlp"
all_args.env_name = "Patrolling"
all_args.user_name = "ideas-mas"

all_args.num_agents = 2
all_args.agent_speed = 10.0
all_args.action_method = "full"
all_args.observe_method = "pyg"
all_args.observe_method_global = "adjacency"
all_args.observation_radius = 200.0
all_args.observation_bitmap_size = 40
all_args.alpha = 0.0
all_args.beta = 1.0
# all_args.reward_method_terminal = "averageAverage"
all_args.reward_method_terminal = "none"
all_args.reward_interval = 1

all_args.graph_name = "4nodes"
all_args.graph_file = f"patrolling_zoo/env/{all_args.graph_name}.graph"
all_args.num_env_steps = 10e5 * 1 #total number of steps
all_args.episode_length = 100 #number of steps in a training episode
all_args.max_cycles = all_args.episode_length #number of steps in an environment episode

all_args.algorithm_name = "mappo"
all_args.use_gnn_policy = True
all_args.use_gnn_mlp_policy = False
all_args.use_recurrent_policy = True
all_args.use_naive_recurrent_policy = False
all_args.use_centralized_V = True
all_args.use_gae = False
all_args.use_gae_amadm = True
all_args.share_policy = False
all_args.sep_share_policy = False
all_args.share_reward = False
all_args.skip_steps_sync = False
all_args.skip_steps_async = True
all_args.use_ReLU = True
# all_args.lr = 1e-4
# all_args.entropy_coef = 0.1
all_args.hidden_size = 1024

all_args.n_rollout_threads = 5
all_args.save_interval = 10
all_args.cuda = True
all_args.cuda_idx = 4

all_args.use_wandb = True

In [3]:
main([], parsed_args = all_args)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


u are choosing to use mappo, we set use_recurrent_policy & use_naive_recurrent_policy to be False
choose to use gpu...


wandb: Currently logged in as: anthony-goeckner (ideas-mas). Use `wandb login --relogin` to force relogin



 Env Patrolling Algo mappo Exp gnnMultiLayerNoMlp updates 0/2000 episodes, total num timesteps 500/1000000.0, FPS 65.

average episode rewards is -4.207987308502197 and idleness is 5.8

 Env Patrolling Algo mappo Exp gnnMultiLayerNoMlp updates 1/2000 episodes, total num timesteps 1000/1000000.0, FPS 68.

average episode rewards is -4.692533016204834 and idleness is 6.8

 Env Patrolling Algo mappo Exp gnnMultiLayerNoMlp updates 2/2000 episodes, total num timesteps 1500/1000000.0, FPS 70.

average episode rewards is -4.630892276763916 and idleness is 7.1

 Env Patrolling Algo mappo Exp gnnMultiLayerNoMlp updates 3/2000 episodes, total num timesteps 2000/1000000.0, FPS 72.

average episode rewards is -4.695237159729004 and idleness is 6.7

 Env Patrolling Algo mappo Exp gnnMultiLayerNoMlp updates 4/2000 episodes, total num timesteps 2500/1000000.0, FPS 68.

average episode rewards is -28.018238067626953 and idleness is 25.05

 Env Patrolling Algo mappo Exp gnnMultiLayerNoMlp updates 5/20

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


actor_grad_norm,█▁▁▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
agent0/actor_grad_norm,▇▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
agent0/critic_grad_norm,▆▂▆▂▅▁▁▂█▁▆▁▁▁▁▁▂▂▂▁▁▁▂▁▄▁▃▂▁▁▁▁▂▁▂▂▁▁▁▂
agent0/dist_entropy,▆▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
agent0/policy_loss,▂▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
agent0/ratio,█▇██▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
agent0/value_loss,▃█▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
agent1/actor_grad_norm,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁
agent1/critic_grad_norm,▆▂▆▂▅▁▁▂█▁▆▁▁▁▁▁▂▂▂▁▁▁▂▁▄▁▃▂▁▁▁▁▂▁▂▂▁▁▁▂
agent1/dist_entropy,▂▁▃█▁▁▂▂▁▁▁▂▁▁▂▁▁▁▂▁▁▁▁▁▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁▂
agent1/policy_loss,█▁▂▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
